In [ ]:
!pip install -q openai anthropic cohere scikit-learn scipy matplotlib pandas supabase requests langchain_openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 6.8 MB/s eta 0:00:00


In [ ]:
import openai
import anthropic
import cohere
import requests
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from scipy.stats import f_oneway
from supabase import create_client
import os
import time
from datetime import datetime
from google.colab import files
from IPython.display import display, HTML, clear_output
from langchain_openai import OpenAIEmbeddings  # New package as per deprecation warning
import seaborn as sns

ModuleNotFoundError: No module named 'anthropic'

In [ ]:
#REading the API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAI')
supabase_key = userdata.get('Supabase_key') #vector store
MISTRAL_API_KEY = userdata.get('Mistral')
CLAUDE_API_KEY = userdata.get('Anthropic')
COHERE_API_KEY = userdata.get('Cohere')
DEEPSEEK_API_KEY = userdata.get('Deepseek_new')
GEMINI_API_KEY = userdata.get('Gemini')

In [ ]:
 ### Supabase Credentials
SUPABASE_URL = "https://rrjbrtbsvdoxndchvchq.supabase.co"
SUPABASE_KEY = supabase_key

# Initialize API clients
openai.api_key = OPENAI_API_KEY
anthropic_client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
cohere_client = cohere.Client(COHERE_API_KEY)

# Create Supabase client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:

# @title ⚙️ Step 4: Configure Experiment
# @markdown Select which models to include in your experiment:

use_claude = True # @param {type:"boolean"}
use_openai = True # @param {type:"boolean"}
use_mistral = True # @param {type:"boolean"}
use_cohere = True # @param {type:"boolean"}
use_deepseek = True # @param {type:"boolean"}

# @markdown Define the test questions:
question_text = "What is the Cafeteria plan?" # @param {type:"string"}
question_list = [
   "What are the Cafeteria plan benefits?", # 3989
   "How can I modify a page on dtsc.be?", # 3999
   "How do I access the shared library?", # 4001
   "Who is responsible if I receive a traffic fine?", # 4008
   "How do I create a teams meeting?", # 4017
   "What activities are included in the DTeam Spirit Challenge?", # 4020
   "Can you tell me about Officient employee self-service?", # 4026
   "Tell me about the Onboarding To Do List?", # 4027
   "How can I activate music streaming on my company car?", # 4043
   "How to access a shared mailbox?" # 4047
]

num_iterations = 50 # @param {type:"slider", min:1, max:50, step:1}

# @markdown Advanced settings:
top_k_docs = 2 # @param {type:"slider", min:1, max:10, step:1}
temperature = 0.7 # @param {type:"slider", min:0, max:1, step:0.1}
match_threshold = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}

# Define LLM models
llm_models = {}

if use_claude:
    llm_models["Claude"] = {"provider": "anthropic", "model": "claude-3-7-sonnet-latest"}
if use_openai:
    llm_models["GPT"] = {"provider": "openai", "model": "gpt-4o"}
if use_mistral:
    llm_models["Mistral"] = {"provider": "mistral", "model": "mistral-large-latest"}
if use_cohere:
    llm_models["Cohere"] = {"provider": "cohere", "model": "command-a-03-2025"}
if use_deepseek:
    llm_models["Deepseek"] = {"provider": "deepseek", "model": "deepseek-chat"}

In [ ]:
def get_embedding(text):
    """Get embeddings for a text using OpenAI's embedding model"""
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,model="text-embedding-3-small")  # Use the appropriate model
    response = embeddings.embed_query(text)  # Correctly call the method to generate embeddings
    embedding = response
    return embedding


In [ ]:
# Generate the list of test questions
questions = question_list * num_iterations

In [ ]:
questions

In [ ]:
# @title 🔍 Step 5: Vector Search Function
def retrieve_context(query, top_k=top_k_docs):
    """Retrieve relevant documents from Supabase vector store using OpenAI embeddings and RPC with filter support"""
    try:
        # Get embedding for the query using OpenAI
        query_embedding = get_embedding(query)

        # Call the custom Supabase function via RPC with filter
        response = supabase.rpc(
            'match_documents',
            {
                'query_embedding': query_embedding,
                'match_count': top_k,
                'filter': {}  # You can pass filters here later if needed
            }
        ).execute()

        if not response.data or len(response.data) == 0:
            return "No relevant context found in the knowledge base."

        # Format retrieved documents as context
        context = "Context from knowledge base:\n\n"
        for i, doc in enumerate(response.data):
            content = doc.get('content', '')
            source = doc.get('source', 'Unknown source')
            context += f"Document {i+1}: {content}\nSource: {source}\n\n"

        return context

    except Exception as e:
        print(f"Error retrieving from vector store: {e}")
        return "Error retrieving context from knowledge base."


In [ ]:
#sanity check
query = "How can I access shared mailbox?"
context = retrieve_context(query)
print(context)

In [ ]:
# @title 🧠 Step 6: Query LLM with RAG for Different Providers
def query_llm_with_rag(llm_config, query):
    provider = llm_config["provider"]
    model = llm_config["model"]

    # Retrieve relevant context from Supabase
    context = retrieve_context(query)

    # Construct RAG prompt with retrieved context
    rag_prompt = f"""
{context}

Based on the above context from our knowledge base, please answer the following question:
{query}

If the context doesn't contain relevant information to answer the question,
please say so and answer based on your general knowledge.
"""

    try:
        # Call the appropriate API based on provider
        if provider == "openai":
            # Updated OpenAI client API call for v1.0+
            response = openai.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": rag_prompt}],
                temperature=temperature
            )
            return response.choices[0].message.content.strip()

        elif provider == "anthropic":
            response = anthropic_client.messages.create(
                model=model,
                messages=[{"role": "user", "content": rag_prompt}],
                max_tokens=1000,
                temperature=temperature
            )
            return response.content[0].text

        elif provider == "cohere":
            response = cohere_client.chat(
                message=rag_prompt,
                model=model,
                temperature=temperature
            )
            return response.text

        elif provider == "mistral":
            headers = {
                "Content-Type": "application/json",
                "Accept": "application/json",
                "Authorization": f"Bearer {MISTRAL_API_KEY}"
            }
            payload = {
                "model": model,
                "messages": [{"role": "user", "content": rag_prompt}],
                "temperature": temperature
            }
            response = requests.post(
                "https://api.mistral.ai/v1/chat/completions",
                headers=headers,
                json=payload
            )
            return response.json()["choices"][0]["message"]["content"].strip()

        elif provider == "deepseek":
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {DEEPSEEK_API_KEY}"
            }
            payload = {
                "model": model,
                "messages": [{"role": "user", "content": rag_prompt}],
                "temperature": temperature
            }
            response = requests.post(
                "https://api.deepseek.com/v1/chat/completions",
                headers=headers,
                json=payload
            )
            return response.json()["choices"][0]["message"]["content"].strip()

        else:
            return f"Error: Unsupported provider {provider}"

    except Exception as e:
        return f"Error with {provider} ({model}): {str(e)}"

In [ ]:
#Sanity checks
query = "What is a cafeteria plan?"
response = query_llm_with_rag(llm_models["Deepseek"], query)
print(response)

Based on the provided context, a **Cafeteria Plan** is an employee benefits program that allows employees to customize their compensation package by choosing from a menu of benefits tailored to their individual needs and preferences.  

### Key Features of the Cafeteria Plan at DTSC:  
1. **Flexibility** – Employees can select benefits that align with their personal and financial goals rather than a one-size-fits-all approach.  
2. **Options Available** – Choices include pension savings, IT device coverage, hospitalization insurance, extra vacation days, gym memberships, and more.  
3. **Annual Selection Process** – Around **November 25th**, employees receive a document listing options and have **three weeks** to submit their preferences.  
4. **Advantages** – The plan promotes empowerment, work-life balance, financial planning, and job satisfaction while aiding in talent retention.  

This system is designed to give employees control over their benefits, ensuring they receive what mat

In [ ]:

# @title 📈 Step 7: Drift Calculation
def compute_response_drift_score(responses):
    if len(responses) < 2:
        return 0, []

    base_embedding = get_embedding.encode(responses[0])
    scores = []
    for r in responses[1:]:
        emb = get_embedding.encode(r)
        sim = cosine_similarity([base_embedding], [emb])[0][0]
        drift_score = 1 - sim
        scores.append(drift_score)
    return np.mean(scores), scores

In [ ]:

def run_experiment():
    # Track unique questions and their repetitions
    unique_questions = {}  # Maps question text to question base ID
    question_counts = {}   # Tracks how many times each unique question has been asked

    response_logs_by_question = {}
    df_data = []

    for i, question in enumerate(questions):
        # Check if this is a repetition of a previous question
        if question in unique_questions:
            base_question_id = unique_questions[question]
            question_counts[question] += 1
            repetition_num = question_counts[question]
            question_id = f"{base_question_id}_{repetition_num}"  # e.g., "q1_2" for second repetition
        else:
            # First time seeing this question
            base_question_id = f"q{len(unique_questions) + 1}"
            unique_questions[question] = base_question_id
            question_counts[question] = 1
            repetition_num = 1
            question_id = f"{base_question_id}_{repetition_num}"  # e.g., "q1_1" for first instance

        print(f"\n Iteration {i+1}/{len(questions)} — Question: {question} (ID: {question_id})")

        # Initialize the entry for this question instance
        response_logs_by_question[question_id] = {
            "question_text": question,
            "base_question_id": base_question_id,
            "repetition_num": repetition_num,
            "responses": {llm: [] for llm in llm_models}
        }

        for llm_name, config in llm_models.items():
            print(f"\n {llm_name} is generating a response...")
            try:
                response = query_llm_with_rag(config, question)
                #print(f" {llm_name} says:\n{response}\n")

                # Store the response in the nested structure
                response_logs_by_question[question_id]["responses"][llm_name].append(response)

                # Add row to DataFrame data
                df_data.append({
                    "question_id": question_id,
                    "base_question_id": base_question_id,
                    "repetition_num": repetition_num,
                    "question_text": question,
                    "llm": llm_name,
                    "response": response,
                    "iteration": i+1
                })

                # Add a small delay to avoid rate limits
                time.sleep(1)

            except Exception as e:
                print(f" Error with {llm_name}: {e}")

    # Create DataFrame from collected data
    response_df = pd.DataFrame(df_data)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"llm_rag_drift_log_{timestamp}.csv"
    response_df.to_csv(filename, index=False)
    print(f"\n✅ Log saved to '{filename}'")

    # For Google Colab: Allow downloading the file
    try:
        files.download(filename)
    except:
        print(f"To download the log file, use the Files panel on the left sidebar.")

    return response_logs_by_question, response_df

In [ ]:
#sanity check
response_logs, response_df = run_experiment()

In [ ]:
response_df

**What this code accomplishes:**

This code helps evaluate how consistent different LLMs are over multiple responses. A lower drift score means the LLM is giving more consistent responses across iterations, while higher drift means the responses are changing more significantly.
The ANOVA test determines whether the different LLMs have statistically different levels of consistency (drift). The visualization helps to see trends in drift over time and compare performance between models.

**Function: compute_response_drift_score**

This function measures how much consecutive responses from the same LLM differ from each other:

It takes a list of responses as input
If there are fewer than 2 responses, it returns 0 (no drift can be calculated)
It converts all responses into embeddings (vector representations) using a function called get_embedding
It calculates the cosine similarity between consecutive embeddings
It converts similarity to drift by subtracting from 1 (so higher values mean more difference)
It returns both the average drift and a list of all drift scores


**Function: analyze_drift**

This function analyzes drift across multiple LLMs:

It processes each LLM's responses to calculate drift scores
It checks if there are enough responses (at least 2) to calculate drift
It performs ANOVA (Analysis of Variance) test to check if different LLMs have statistically significant differences in drift
It prints results including average drift scores for each LLM and ANOVA results
It creates and saves a plot showing drift over time for each LLM

In [ ]:
def analyze_drift_for_repeated_questions_df(response_df):
    """
    Analyze drift across repeated instances of the same questions using DataFrame
    """
    print("\n📊 Calculating Response Drift Scores for Repeated Questions...")

    # Extract unique LLM models from df
    llm_models = response_df['llm'].unique()

    # Group question IDs by base question
    questions_by_base = {}
    for _, row in response_df.drop_duplicates(['question_id']).iterrows():
        base_id = row['base_question_id']
        q_id = row['question_id']
        if base_id not in questions_by_base:
            questions_by_base[base_id] = []
        questions_by_base[base_id].append(q_id)

    # Sort question IDs within each base group by repetition number
    for base_id in questions_by_base:
        questions_by_base[base_id].sort(key=lambda q_id: int(q_id.split('_')[1]))

    # Initialize drift logs
    drift_logs = {llm: {base_id: [] for base_id in questions_by_base} for llm in llm_models}
    drift_df_data = []

    # Collect all drift scores by LLM for ANOVA
    all_drift_by_llm = {llm: [] for llm in llm_models}

    for base_id, q_ids in questions_by_base.items():
        if len(q_ids) < 2:
            print(f"Base question {base_id} doesn't have multiple repetitions, skipping drift analysis")
            continue

        # Get question text from first instance
        question_text = response_df[response_df['question_id'] == q_ids[0]]['question_text'].iloc[0]
        print(f"\nAnalyzing drift for question: '{question_text[:50]}...' (Base ID: {base_id})")

        for llm in llm_models:
            # Get all responses for this LLM across repetitions of this question
            responses = []
            for q_id in q_ids:
                llm_responses = response_df[(response_df['question_id'] == q_id) & (response_df['llm'] == llm)]['response'].tolist()
                if llm_responses:
                    responses.append(llm_responses[0])

            if len(responses) < 2:
                print(f"  {llm}: Not enough responses across repetitions to calculate drift")
                continue

            # Calculate drift between consecutive repetitions
            avg_drift, all_drifts = compute_response_drift_score(responses)
            drift_logs[llm][base_id] = all_drifts

            # Collect drift scores for ANOVA
            all_drift_by_llm[llm].extend(all_drifts)

            print(f"  {llm} - Drift between repetitions: {', '.join([f'{d:.4f}' for d in all_drifts])}")
            print(f"  {llm} - Average drift: {avg_drift:.4f}")

            # Add to DataFrame data
            for i, drift_score in enumerate(all_drifts):
                rep_from = i + 1
                rep_to = i + 2
                drift_df_data.append({
                    "base_question_id": base_id,
                    "question_text": question_text,
                    "llm": llm,
                    "from_repetition": rep_from,
                    "to_repetition": rep_to,
                    "drift_score": drift_score,
                })

    # Perform ANOVA test to compare LLMs
    print("\n📊 Statistical Analysis (ANOVA):")
    valid_llms = [llm for llm in llm_models if len(all_drift_by_llm[llm]) > 0]

    if len(valid_llms) > 1:
        # Prepare data for ANOVA test
        anova_data = [all_drift_by_llm[llm] for llm in valid_llms]

        # Make sure all groups have data
        if all(len(data) > 0 for data in anova_data):
            try:
                from scipy.stats import f_oneway
                anova_result = f_oneway(*anova_data)

                print(f"ANOVA F-statistic: {anova_result.statistic:.4f}, p-value: {anova_result.pvalue:.4f}")

                if anova_result.pvalue < 0.05:
                    print("🎯 Statistically significant difference in drift detected between LLMs!")

                    # If significant, perform post-hoc tests to see which models differ
                    try:
                        from statsmodels.stats.multicomp import pairwise_tukeyhsd
                        import numpy as np

                        # Prepare data for Tukey's test
                        all_values = []
                        model_labels = []

                        for i, llm in enumerate(valid_llms):
                            all_values.extend(all_drift_by_llm[llm])
                            model_labels.extend([llm] * len(all_drift_by_llm[llm]))

                        # Perform Tukey's HSD test
                        tukey_result = pairwise_tukeyhsd(np.array(all_values), model_labels, alpha=0.05)
                        print("\nTukey's HSD Post-hoc Test:")
                        print(tukey_result)
                    except ImportError:
                        print("Note: statsmodels not available for post-hoc tests")
                else:
                    print("✅ No significant difference in drift detected between LLMs")
            except ImportError:
                print("Note: scipy.stats not available for ANOVA test")
        else:
            print("Not enough data points for all LLMs to perform ANOVA test")
    else:
        print("Need at least two LLMs with data to perform statistical comparison")

    # Create DataFrame for drift analysis
    if drift_df_data:
        drift_df = pd.DataFrame(drift_df_data)
        drift_df.to_csv("drift_analysis.csv", index=False)

        # Create visualizations
        import matplotlib.pyplot as plt
        import seaborn as sns

        plt.figure(figsize=(12, 8))

        # For each base question, create a subplot
        num_base_questions = len(questions_by_base)
        cols = min(2, num_base_questions)
        rows = (num_base_questions + cols - 1) // cols  # Ceiling division

        for i, (base_id, _) in enumerate(questions_by_base.items()):
            if len(questions_by_base[base_id]) < 2:
                continue

            plt.subplot(rows, cols, i+1)

            for llm in llm_models:
                if base_id in drift_logs[llm] and drift_logs[llm][base_id]:
                    x = range(1, len(drift_logs[llm][base_id])+1)
                    plt.plot(x, drift_logs[llm][base_id], marker='o', label=llm)

            question_text = response_df[response_df['base_question_id'] == base_id]['question_text'].iloc[0]
            plt.title(f"Q{base_id[-1]}: {question_text[:30]}...")
            plt.xlabel("Transition Between Repetitions")
            plt.ylabel("Drift Score")
            plt.grid(True, alpha=0.3)
            plt.legend()

        plt.tight_layout()
        plt.savefig("repeated_questions_drift.png")
        plt.show()

        # Summary bar chart of average drift by question and LLM
        avg_drift_data = []
        for llm in llm_models:
            for base_id in questions_by_base:
                if base_id in drift_logs[llm] and drift_logs[llm][base_id]:
                    avg_drift = sum(drift_logs[llm][base_id]) / len(drift_logs[llm][base_id])
                    question_text = response_df[response_df['base_question_id'] == base_id]['question_text'].iloc[0]
                    avg_drift_data.append({
                        "base_question_id": base_id,
                        "question_short": question_text[:20] + "...",
                        "llm": llm,
                        "avg_drift": avg_drift
                    })

        if avg_drift_data:
            avg_drift_df = pd.DataFrame(avg_drift_data)

            plt.figure(figsize=(14, 6))
            sns.barplot(x="base_question_id", y="avg_drift", hue="llm", data=avg_drift_df)
            plt.title("Average Drift by Question and LLM")
            plt.xlabel("Question")
            plt.ylabel("Average Drift Score")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig("avg_drift_by_question.png")
            plt.show()

    return drift_logs, drift_df if drift_df_data else None


def compute_response_drift_score(responses):
    """
    Calculate drift between consecutive responses

    Parameters:
    - responses: List of response texts ordered by repetition

    Returns:
    - avg_drift: Average drift score across all transitions
    - drift_scores: List of individual drift scores between consecutive responses
    """
    if len(responses) < 2:
        return 0, []

    # You'll need to implement your drift calculation logic here
    # For example, using cosine similarity between embeddings

    drift_scores = []
    for i in range(len(responses) - 1):
        # Calculate similarity between consecutive responses
        # Here's a placeholder - replace with your actual similarity calculation
        try:
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity

            # Using TF-IDF and cosine similarity as a measure of drift
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform([responses[i], responses[i+1]])
            similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

            # Convert similarity to drift (1 - similarity)
            drift = 1 - similarity
            drift_scores.append(drift)
        except ImportError:
            # Fallback to a simple text similarity if sklearn not available
            import difflib
            similarity = difflib.SequenceMatcher(None, responses[i], responses[i+1]).ratio()
            drift = 1 - similarity
            drift_scores.append(drift)

    avg_drift = sum(drift_scores) / len(drift_scores) if drift_scores else 0
    return avg_drift, drift_scores

In [ ]:
response_df = pd.read_csv("/content/llm_rag_drift_log_20250410_015221.csv")

In [ ]:
#sanity check
drift_logs, drift_df = analyze_drift_for_repeated_questions_df(response_df)


In [ ]:
drift_df

,base_question_id,question_text,llm,from_repetition,to_repetition,drift_score
0,q1,What are the Cafeteria plan benefits?,Claude,1,2,0.089315
1,q1,What are the Cafeteria plan benefits?,Claude,2,3,0.064863
2,q1,What are the Cafeteria plan benefits?,GPT,1,2,0.066217
3,q1,What are the Cafeteria plan benefits?,GPT,2,3,0.028247
4,q1,What are the Cafeteria plan benefits?,Mistral,1,2,0.067029
5,q1,What are the Cafeteria plan benefits?,Mistral,2,3,0.053908
6,q1,What are the Cafeteria plan benefits?,Cohere,1,2,0.023514
7,q1,What are the Cafeteria plan benefits?,Cohere,2,3,0.026278
8,q1,What are the Cafeteria plan benefits?,Deepseek,1,2,0.081906
9,q1,What are the Cafeteria plan benefits?,Deepseek,2,3,0.081304


In [ ]:


# @title 📊 Step 10: Feedback Visualization
def visualize_feedback(feedback_logs):
    # Convert feedback to numeric values
    feedback_values = {"positive": 1, "neutral": 0, "negative": -1, "error": None}

    feedback_data = {}
    for llm, feedback in feedback_logs.items():
        feedback_data[llm] = [feedback_values.get(f) for f in feedback if feedback_values.get(f) is not None]

    # Plot average feedback scores
    plt.figure(figsize=(10, 6))
    avg_scores = {llm: np.mean(scores) if scores else 0 for llm, scores in feedback_data.items()}

    models = list(avg_scores.keys())
    scores = list(avg_scores.values())

    # Sort by score
    sorted_indices = np.argsort(scores)
    sorted_models = [models[i] for i in sorted_indices]
    sorted_scores = [scores[i] for i in sorted_indices]

    bars = plt.barh(sorted_models, sorted_scores, color=['red' if s < 0 else 'green' if s > 0 else 'gray' for s in sorted_scores])
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.xlabel("Average Feedback Score (-1: Negative, 0: Neutral, 1: Positive)")
    plt.title("Average User Feedback by LLM")
    plt.xlim(-1.1, 1.1)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig("feedback_analysis.png")
    plt.show()

In [ ]:
#sanity check
visualize_feedback(feedback_logs)

In [ ]:


# @title 💾 Step 11: Export Logs
def export_logs(log_df):
    df = pd.DataFrame(log_df)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"llm_rag_drift_log_{timestamp}.csv"
    df.to_csv(filename, index=False)
    print(f"\n✅ Log saved to '{filename}'")

    # For Google Colab: Allow downloading the file
    try:
        files.download(filename)
    except:
        print(f"To download the log file, use the Files panel on the left sidebar.")

    return df


In [ ]:
#sanity check
df_log = export_logs(log_df)


✅ Log saved to 'llm_rag_drift_log_20250408_113449.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# @title 📝 Step 12: Generate Summary Report
def generate_report(df, drift_logs):
    print("\n📋 Generating Summary Report...")

    # Calculate statistics per model
    model_stats = {}
    for llm in llm_models:
       # Calculate average drift
        avg_drift = np.mean(drift_logs[llm]) if len(drift_logs[llm]) > 0 else 0

        model_stats[llm] = {
            'provider': llm_models[llm]['provider'],
            'model': llm_models[llm]['model'],
            'avg_drift': avg_drift
        }

    # Create summary dataframe
    summary_df = pd.DataFrame.from_dict(model_stats, orient='index').reset_index()
    summary_df = summary_df.rename(columns={'index': 'llm'})

    # Sort by feedback score (descending)
    summary_df = summary_df.sort_values('feedback_score', ascending=False)

    # Save summary
    summary_filename = "llm_performance_summary.csv"
    summary_df.to_csv(summary_filename, index=False)
    print(f"✅ Summary report saved to '{summary_filename}'")

    # For Google Colab: Allow downloading the file
    try:
        files.download(summary_filename)
    except:
        print(f"To download the summary file, use the Files panel on the left sidebar.")

    # Print summary
    print("\n📊 LLM Performance Summary:")
    display(summary_df[['llm', 'avg_drift']])

    return summary_df

In [ ]:
#sanity checks
summary_df = generate_report(df_log, drift_logs)

In [ ]:
def main():
    print("🔄 Starting Multi-LLM RAG Drift Analysis Experiment")

    # Check if API keys are provided
    missing_keys = []
    if use_claude and not CLAUDE_API_KEY: missing_keys.append("Anthropic (Claude)")
    if use_openai and not OPENAI_API_KEY: missing_keys.append("OpenAI")
    if use_mistral and not MISTRAL_API_KEY: missing_keys.append("Mistral")
    if use_cohere and not COHERE_API_KEY: missing_keys.append("Cohere")
    if use_deepseek and not DEEPSEEK_API_KEY: missing_keys.append("DeepSeek")

    if missing_keys:
        print(f"❌ Missing API keys for: {', '.join(missing_keys)}")
        print("Please provide the required API keys and run again.")
        return

    # Check if any models are selected
    if not llm_models:
        print("❌ No LLM models selected. Please select at least one model.")
        return

    # Check if Supabase connection works
    if not SUPABASE_URL or not SUPABASE_KEY:
        print("❌ Missing Supabase credentials. Please provide URL and key.")
        return

    try:
        test_query = "test connection"
        test_context = retrieve_context(test_query, top_k=1)
        print("✅ Supabase connection successful")
    except Exception as e:
        print(f"❌ Supabase connection failed: {e}")
        print("Please check your Supabase credentials and try again.")
        return

    # Print active models
    print("\n🤖 Active models for this experiment:")
    for llm, config in llm_models.items():
        print(f"  - {llm} ({config['provider']}: {config['model']})")

    # Run the experiment
    start_time = time.time()
    response_logs = run_experiment()
    data = pd.DataFrame(response_logs)
    data.to_csv("response_logs.csv", index=False)
    drift_logs = analyze_drift(response_logs)
    summary_df = generate_report(drift_logs)

    # Calculate and display execution time
    execution_time = time.time() - start_time
    print(f"\n✅ Experiment completed in {execution_time:.2f} seconds")

    return summary_df



In [ ]:
# Execute the main function if this script is run directly
if __name__ == "__main__":
    main()

🔄 Starting Multi-LLM RAG Drift Analysis Experiment
✅ Supabase connection successful

🤖 Active models for this experiment:
  - Claude (anthropic: claude-3-7-sonnet-latest)
  - GPT (openai: gpt-4o-mini)
  - Mistral (mistral: mistral-large-latest)
  - Cohere (cohere: command-r-plus)
  - Deepseek (deepseek: deepseek-chat)

📌 Iteration 1/5 — Question: What is the Cafeteria plan?

🤖 Claude is generating a response...
🗨️ Claude says:
Based on the provided context, the Cafeteria Plan is part of the employee benefits program at DT Services and Consulting. It's designed to allow employees to tailor their benefits package according to their individual needs and preferences, rather than having a one-size-fits-all approach.

The plan enables employees to choose from a menu of different benefits, putting them "in the driver's seat" of their compensation package. Employees receive a document around November 25th each year with options to choose from, and they have approximately three weeks to complet

KeyboardInterrupt: Interrupted by user